In [ ]:
import os
from os import path

# Third-party
from astropy.constants import G
from astropy.io import fits, ascii
from astropy.stats import median_absolute_deviation
from astropy.table import Table, QTable, join
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import tqdm
from scipy.stats import beta, binned_statistic

from thejoker import JokerSamples

from twoface.config import TWOFACE_CACHE_PATH
from twoface.samples_analysis import MAP_sample
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, NessRG,
                        StarResult, Status, JokerRun)
from twoface.plot import plot_two_panel, plot_phase_fold
from twoface.mass import get_m2_min, mf, period_at_surface
from twoface.log import log as logger
logger.setLevel(100)

### TODO: describe Need to download catalogs from `TwoFace`

In [ ]:
plot_path = '../paper/figures/'
table_path = '../data/'

In [ ]:
unimodal = QTable.read(path.join(table_path, 'highK-unimodal.fits'))
clean_uni = unimodal[unimodal['clean_flag'] == 0]
high_logg = clean_uni[clean_uni['LOGG'] > 2]
print('{0} unimodal, {1} clean, {2} high logg'
      .format(len(unimodal), len(clean_uni), len(high_logg)))

In [ ]:
# bins = np.linspace(0, 3.1, 15)
# res15 = binned_statistic(np.log10(cat['P'].value), values=cat['e'], bins=bins,
#                          statistic=lambda x: np.percentile(x, 15))
# res50 = binned_statistic(np.log10(cat['P'].value), values=cat['e'], bins=bins,
#                          statistic=lambda x: np.percentile(x, 50))
# res85 = binned_statistic(np.log10(cat['P'].value), values=cat['e'], bins=bins,
#                          statistic=lambda x: np.percentile(x, 85))

# binc = (res15.bin_edges[:-1]+res15.bin_edges[1:])/2
# plt.plot(binc, res15.statistic)
# plt.plot(binc, res50.statistic)
# plt.plot(binc, res85.statistic)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 8.5), sharex=True, sharey=True)

grid = np.arange(2, 3.5+0.1, 0.3)
for i, (l, r) in enumerate(zip(grid, grid+0.5)):
    mask = (clean_uni['LOGG'] > l) & (clean_uni['LOGG'] <= r)
    subcat = clean_uni[mask]
    axes.flat[i].errorbar(subcat['P'].value, subcat['e'], 
                          xerr=subcat['P_err'].value, yerr=subcat['e_err'],
                          marker='o', linestyle='none', alpha=0.8)
    axes.flat[i].axvline(10., zorder=-10, color='#aaaaaa', linestyle='--')
    axes.flat[i].axvline(100., zorder=-10, color='#aaaaaa', linestyle='--')
    axes.flat[i].text(1, 0.95, r'${0:.2f} < \log g \leq {1:.2f}$'.format(l, r), 
                      ha='left', va='top', fontsize=16, backgroundcolor='w')
    
    
axes.flat[0].set_xlim(0.8, 1800)
axes.flat[0].set_ylim(-0.05, 1.)
axes.flat[0].set_xscale('log')

axes[1, 1].set_xlabel('period, $P$ [day]')
axes[0, 0].set_ylabel('eccentricity, $e$')
axes[1, 0].set_ylabel('eccentricity, $e$')

fig.tight_layout()

# fig.savefig(path.join(plot_path, 'P-e-grid.pdf'))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10., 4.5), sharex=True)

P_surface = period_at_surface(1.36*u.Msun, high_logg['LOGG'], 
                              high_logg['e'], 0.5*u.Msun)

cs = axes[0].scatter(high_logg['P']/P_surface, high_logg['e'],
                     c=high_logg['LOGG'], alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=2, vmax=4, 
                     cmap='viridis')

axes[0].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[0].set_xlabel(r'$P / P_{\rm surface}$')
axes[1].set_xlabel(r'$P / P_{\rm surface}$')
axes[0].set_ylabel(r'eccentricity, $e$')

axes[0].set_ylim(-0.05, 1)
axes[0].set_xlim(0.4, 2000)
axes[1].set_xscale('log')
axes[0].yaxis.set_ticks(np.arange(0, 1+0.2, 0.2))

# ---

cs = axes[1].scatter(high_logg['P']/P_surface, high_logg['K'],
                     c=high_logg['LOGG'], alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=2, vmax=4, 
                     cmap='viridis')

axes[1].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[1].set_ylim(1, 250)
axes[1].set_yscale('log')
axes[1].set_ylabel('amplitude, $K$ [{0:latex_inline}]'.format(u.km/u.s))

cax = fig.add_axes([0.87, 0.185, 0.02, 0.76])
cb = fig.colorbar(cs, cax=cax)
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
cb.set_label(r'$\log g$', labelpad=10)
cb.solids.set_rasterized(True) 
cb.solids.set_edgecolor('face')
cb.set_ticks(np.arange(2, 4+0.1, 0.5))

fig.tight_layout()
fig.subplots_adjust(right=0.85, wspace=0.35)

# fig.savefig(path.join(plot_path, 'P-e-K.pdf'))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

bins = np.arange(-0.1, 1+0.1, 0.1)
bin_c = (bins[:-1]+bins[1:]) / 2
# cut = high_logg['P'] < 10*u.day

P_surface = period_at_surface(1.36*u.Msun, high_logg['LOGG'], 
                              high_logg['e'], 0.5*u.Msun)
cut = (high_logg['P']/P_surface) < 8

# ax.hist(high_logg['e'][cut], bins=bins, 
#         alpha=0.5, normed=True, rasterized=True,
#         label=r'$P < 20\,{\rm d}$' + '  $(N = {0})$'.format(cut.sum()));
# ax.hist(high_logg['e'][~cut], bins=bins, 
#         alpha=0.5, normed=True, rasterized=True,
#         label=r'$P \geq 20\,{\rm d}$' + '  $(N={0})$'.format(np.logical_not(cut).sum()));

n, *_ = np.histogram(high_logg['e'][cut], bins=bins, density=True)
ax.plot(bin_c, n, 
        alpha=0.9, drawstyle='steps-mid', marker='', lw=2.,
        label=r'$P < 10\,{\rm d}$' + '  $(N = {0})$'.format(cut.sum()));

n, *_ = np.histogram(high_logg['e'][~cut], bins=bins, density=True)
ax.plot(bin_c, n, 
        alpha=0.9, drawstyle='steps-mid', marker='', lw=2.,
        label=r'$P \geq 10\,{\rm d}$' + '  $(N={0})$'.format(np.logical_not(cut).sum()));

egrid = np.linspace(0, 1, 256)
ax.plot(egrid, beta.pdf(egrid, 0.867, 3.03),
        marker='', linestyle='--', color='k', alpha=0.4,
        label='prior', zorder=-10)

# vals = np.zeros(len(bins)-1)
# for i, l, r in zip(range(len(vals)), bins[:-1], bins[1:]):
#     vals[i] = quad(lambda x: beta.pdf(x, 0.867, 3.03), l, r)[0]
# ax.plot(bin_c, vals * n.sum(), drawstyle='steps-mid',
#         marker='', linestyle='-', color='k', alpha=0.4,
#         label='prior', zorder=-10)

ax.legend(loc='best', fontsize=16)
ax.set_xlabel('eccentricity, $e$')
ax.set_ylabel('$p(e)$')

ax.set_xlim(-0.02, 1.02)
ax.set_yscale('log')
ax.set_ylim(8E-2, 2E1)

fig.tight_layout()

# fig.savefig(path.join(plot_path, 'ecc-dist.pdf'), dpi=250)